In [1]:
import pandas as pd
import numpy as np
import os
import shutil
import glob

In [2]:
# Please change this with the directory of the folder that contains the csv files downloaded
# from your Drive
nameOfCSVFolderDir = "/home/milto/Documents/fieldData/gdrivefolder_Sentinel2"
nameOfCSVFolderDir = "/home/milto/Documents/fieldData/WORKS1"
# File exported that contains the column "" that will be used for merging the files
fieldDataDir       ="./.ipynb_checkpoints/FieldDataWithIdentifiers.csv"
samplingSizeFile   ="./.ipynb_checkpoints/SamplingSize.txt"
samplingSize = 400

# make tmp directories
TmpDir = os.path.join(nameOfCSVFolderDir,"TmpDir")
S2DirMean = os.path.join(nameOfCSVFolderDir,"S2Mean")
S1DirStd  = os.path.join(nameOfCSVFolderDir,"S1Std" )
S2DirStd  = os.path.join(nameOfCSVFolderDir,"S2Std" )

# Read sampling rate
try:
  f = open(samplingSizeFile,"r")
  samplingSize = int(f.readline())
  f.close()
except OSError:
    print("ERROR:", samplingSizeFile," does not exist. Setting sampling size to default = 400")
    samplingSize = 400

if os.path.isdir(TmpDir):
    print ("TmpDir Exist")
    shutil.rmtree(TmpDir)


os.mkdir(TmpDir)
    
ListS1Mean=[]
ListS2Mean=[]
ListS1StdD=[]
ListS2StdD=[]

for file in glob.glob(nameOfCSVFolderDir+"/*.csv"):
    S1Mean="S1_mean.csv"
    S2Mean="S2_mean.csv"
    S1StdD="S1_stdD.csv"
    S2StdD="S2_stdD.csv"
    if(len(file)>11):
        if   (file[len(file)-11:len(file)]==S1Mean):
            ListS1Mean=ListS1Mean+[file]
        elif (file[len(file)-11:len(file)]==S2Mean):
            ListS2Mean=ListS2Mean+[file]
        elif (file[len(file)-11:len(file)]==S1StdD):
            ListS1StdD=ListS1StdD+[file]
        elif (file[len(file)-11:len(file)]==S2StdD):
            ListS2StdD=ListS2StdD+[file]
        elif (file[len(file)-11:len(file)]=="tifiers.csv"): #in case field data are added in the nameOfCSVFolderDir
           print(file , " is suspected to be the field data file")
        else :
            print("WARNING: ", file, " is ignored since it is not recognised as an output of the system")

    #df = pd.read_csv(file)
    #df = df.reindex(sorted(df.columns),axis=1)

ListS1Mean.sort()
ListS1StdD.sort()
ListS2Mean.sort()
ListS2StdD.sort()

if(ListS1Mean!=[] and ListS1Mean[0][len(ListS1Mean[0])-33:len(ListS1Mean[0])-23]!="0000000000"):
    raise Exception("ERROR: csv not sorted correctly or files are missing")
if(ListS1StdD!=[] and ListS1StdD[0][len(ListS1StdD[0])-33:len(ListS1StdD[0])-23]!="0000000000"):
    raise Exception("ERROR: csv not sorted correctly or files are missing")
if(ListS2Mean!=[] and ListS2Mean[0][len(ListS2Mean[0])-33:len(ListS2Mean[0])-23]!="0000000000"):
    raise Exception("ERROR: csv not sorted correctly or files are missing")
if(ListS2StdD!=[] and ListS2StdD[0][len(ListS2StdD[0])-33:len(ListS2StdD[0])-23]!="0000000000"):
    raise Exception("ERROR: csv not sorted correctly or files are missing")

if ListS1Mean!=[] and ListS2Mean!=[] and len(ListS2Mean)!=len(ListS1Mean):
   raise Exception("ERROR: if both Sentinel 1 and Sentinel 2 data were loaded then equal number of csv files should have existed - maybe GEE processing have not finished yet")
if ListS1StdD!=[] and ListS2StdD!=[] and len(ListS2StdD)!=len(ListS1StdD):
   raise Exception("ERROR: if both Sentinel 1 and Sentinel 2 data were loaded then equal number of csv files should have existed - maybe GEE processing have not finished yet")



firstItem = None
if(ListS1Mean!=[]):
    firstItem=ListS1Mean[0]
elif (ListS2Mean!=[]):
    firstItem=ListS2Mean[0]
else :
    raise Exception ("ERROR: no data found. Both Sentinel-1 and Sentinel-2 lists are empty")

tmpDF = pd.read_csv(firstItem)
dfFieldData = pd.read_csv(fieldDataDir)
lenOfFieldData = len(dfFieldData.index)
currentMin = 0
currentMax = samplingSize
fileNames = firstItem[0:len(firstItem)-34] 
head, fileNames = os.path.split(fileNames)
print(fileNames, "&&&&&&&&&&&&&&&&&&&&&&&")


count = 0
while currentMin<lenOfFieldData :
    strCurrentMin = str(currentMin  ).rjust(10,'0')
    strCurrentMax = str(currentMax-1).rjust(10,'0')
    FilenamesWithSampling = fileNames+"_"+strCurrentMin+"_"+strCurrentMax
    print(FilenamesWithSampling)
  

    print(currentMin,currentMax,lenOfFieldData)
    currentMin = currentMax
    currentMax = currentMax + samplingSize
    subsetFieldDF = dfFieldData.iloc[currentMin:currentMax]
    dfMean=subsetFieldDF
    dfStdD=subsetFieldDF
    
    if(count<len(ListS1Mean)):
        tmpDF = pd.read_csv(ListS1Mean[count])
        dfMean = pd.merge(dfMean,tmpDF,on="indexField",how='outer')
        print("MERGING FIELD DATA WITH S1 MEAN")
    if(count<len(ListS2Mean)):
        tmpDF = pd.read_csv(ListS2Mean[count])
        dfMean = pd.merge(dfMean,tmpDF,on="indexField",how='outer')
    
    if(count<len(ListS1StdD)):
        tmpDF = pd.read_csv(ListS1StdD[count])
        dfStdD = pd.merge(dfMean,tmpDF,on="indexField",how='outer')
    if(count<len(ListS2StdD)):
        tmpDF = pd.read_csv(ListS2StdD[count])
        dfStdD = pd.merge(dfMean,tmpDF,on="indexField",how='outer')
    
    dfMean.to_csv(TmpDir+"/"+FilenamesWithSampling+"_Mean.csv", index=False)

    dfStdD.to_csv(TmpDir+"/"+FilenamesWithSampling+"_dfStdD.csv", index=False)

    count = count+1
    #break

print(ListS1Mean)
print(ListS1StdD)
print(ListS2Mean)
print(ListS2StdD)


    #print(df)

#shutil.rmtree(TmpDir)

print("   ***   EXIT   ***")


TmpDir Exist
featurevectorWORK &&&&&&&&&&&&&&&&&&&&&&&
featurevectorWORK_0000000000_0000000599
0 600 1800
featurevectorWORK_0000000600_0000001199
600 1200 1800
featurevectorWORK_0000001200_0000001799
1200 1800 1800
['/home/milto/Documents/fieldData/WORKS1/featurevectorWORK_0000000000_0000000599_S1_mean.csv', '/home/milto/Documents/fieldData/WORKS1/featurevectorWORK_0000000600_0000001199_S1_mean.csv']
[]
[]
[]
   ***   EXIT   ***
